https://keras.io/guides/keras_nlp/getting_started/

In [1]:
!pip install -q --upgrade keras-nlp
!pip install -q --upgrade keras  # Upgrade to Keras 3.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.3.3 which is incompatible.


In [2]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # or "tensorflow" or "torch"

import keras_nlp
import keras

# Use mixed precision to speed up all training in this guide.
keras.mixed_precision.set_global_policy("mixed_float16")

2024-06-16 15:45:39.833868: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-16 15:45:39.833987: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-16 15:45:39.993756: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!# Remove unsupervised examples
!rm -r aclImdb/train/unsup

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  5198k      0  0:00:15  0:00:15 --:--:-- 11.3M


In [5]:
BATCH_SIZE = 16
imdb_train = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=BATCH_SIZE,
)
imdb_test = keras.utils.text_dataset_from_directory(
    "aclImdb/test",
    batch_size=BATCH_SIZE,
)

# Inspect first review
# Format is (review text tensor, label tensor)
print(imdb_train.unbatch().take(1).get_single_element())

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
(<tf.Tensor: shape=(), dtype=string, numpy=b'I saw this feature as part of the Asian American Film Festival in New York and was horrified by the graphic, sado-masochistic, child pornography that I witnessed. The story line is hidden beneath way too many graphic sex scenes - and, not one is in the least bit erotic - sick is the more the feeling. The director seemed to be going for shock value rather the exploring the various levels of why these characters are like this. See it if you can stomach it - I still have flashbacks.'>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)


In [6]:
classifier = keras_nlp.models.BertClassifier.from_preset("bert_tiny_en_uncased_sst2")
# Note: batched inputs expected so must wrap string in iterable
classifier.predict(["I love modular workflows in keras-nlp!"])

Attaching 'metadata.json' from model 'keras/bert/keras/bert_tiny_en_uncased_sst2/4' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/bert/keras/bert_tiny_en_uncased_sst2/4' to your Kaggle notebook...
Attaching 'task.json' from model 'keras/bert/keras/bert_tiny_en_uncased_sst2/4' to your Kaggle notebook...
Attaching 'task.json' from model 'keras/bert/keras/bert_tiny_en_uncased_sst2/4' to your Kaggle notebook...
Attaching 'task.json' from model 'keras/bert/keras/bert_tiny_en_uncased_sst2/4' to your Kaggle notebook...
Attaching 'task.json' from model 'keras/bert/keras/bert_tiny_en_uncased_sst2/4' to your Kaggle notebook...
Attaching 'task.weights.h5' from model 'keras/bert/keras/bert_tiny_en_uncased_sst2/4' to your Kaggle notebook...
Attaching 'task.weights.h5' from model 'keras/bert/keras/bert_tiny_en_uncased_sst2/4' to your Kaggle notebook...
/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optim

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


array([[-1.287,  1.393]], dtype=float16)

In [ ]:
classifier.evaluate(imdb_test)


In [ ]:
classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased",
    num_classes=2,
)
classifier.fit(
    imdb_train,
    validation_data=imdb_test,
    epochs=1,
)

In [ ]:
import tensorflow as tf

preprocessor = keras_nlp.models.BertPreprocessor.from_preset(
    "bert_tiny_en_uncased",
    sequence_length=512,
)

# Apply the preprocessor to every sample of train and test data using `map()`.
# [`tf.data.AUTOTUNE`](https://www.tensorflow.org/api_docs/python/tf/data/AUTOTUNE) and `prefetch()` are options to tune performance, see
# https://www.tensorflow.org/guide/data_performance for details.

# Note: only call `cache()` if you training data fits in CPU memory!
imdb_train_cached = (
    imdb_train.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)
imdb_test_cached = (
    imdb_test.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)

classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased", preprocessor=None, num_classes=2
)
classifier.fit(
    imdb_train_cached,
    validation_data=imdb_test_cached,
    epochs=3,
)